In [1]:
import pandas as pd
import numpy as np
from nltk import word_tokenize
import os
from ast import literal_eval
from nltk.util import ngrams
from nltk.corpus import stopwords
stop=stopwords.words('english')
import spacy
nlp=spacy.load('en_core_web_sm')
from tqdm import tqdm

/Users/mstudio/miniconda3/envs/py3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
path='/Users/mstudio/Library/CloudStorage/Box-Box/coolie/token/'
df=pd.read_csv(path+'/coolie-token.csv', converters={'state':literal_eval, 'context':literal_eval, 'city':literal_eval, 'sent':literal_eval, 'date':literal_eval})

In [7]:
print(df['date'].min(), df['date'].max())

17950630 19631206


In [3]:
df.columns

Index(['state', 'city', 'date', 'lccn', 'sent', 'title'], dtype='object')

In [11]:
def get_ngrams(text, n):
    entire=[]
    n_grams=ngrams(word_tokenize(text), n)
    for i in n_grams:
        entire.append(i)
    return entire

In [3]:
df['context_str']=df['context'].apply(' '.join)

In [4]:
def lemmatization(dataframe:pd.DataFrame()):
    dataframe['alpha']=dataframe['sent'].apply(lambda token_list: [token for token in token_list if token.isalpha()])
    dataframe['stop']=dataframe['alpha'].apply(lambda token_list: [token for token in token_list if token.lower() not in stop])
    dataframe['lemma']=dataframe['stop'].apply(lambda token_list: [token.lemma_ for token in nlp(" ".join(token_list))])
    dataframe.drop(['alpha', 'stop'], axis=1, inplace=True)
    return dataframe
    # dataframe['punct_word']=dataframe['sent'].apply(lambda row: ' '.join([w.lemma_ for w in nlp(row)]))
    # dataframe['sent_stop']=dataframe['punct_word'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    # dataframe['sent_punct']=dataframe['sent_stop'].str.replace('[^\w\s]','')
    # dataframe['sent_lemma']=dataframe['sent_punct'].apply(lambda x: ' '.join(x for x in x.split() if x.isalpha()))
    # dataframe['sent_token']=dataframe['sent_lemma'].apply(word_tokenize)
    # dataframe.drop(['sent_stop', 'sent_punct', 'punct_word'], axis=1, inplace=True)
    # return dataframe

In [15]:
def reuse_df(data:pd.DataFrame()):
    no_reuse=[]
    pair_index1=[]
    pair_index2=[]
    pair_date1=[]
    pair_date2=[]
    pair_text1=[]
    pair_text2=[]
    pair_state1=[]
    pair_state2=[]
    for k in tqdm(range(0,data.shape[0])):
        t=0
        for p in range(t,data.shape[0]):
            t+=1
            if t>k:
                if (k==p) & (len(list(set(data['5-gram'][k]).intersection(data['5-gram'][p]))) == len(data['5-gram'][k])):
                    pass
                elif len(list(set(data['5-gram'][k]).intersection(data['5-gram'][p]))) == 0:
                    pass
                elif len(list(set(data['5-gram'][k]).intersection(data['5-gram'][p]))) >= 3:
                    no_reuse.append(len(list(set(data['5-gram'][k]).intersection(data['5-gram'][p]))))
                    pair_index1.append(k) 
                    pair_index2.append(p)
                    pair_date1.append(data.iloc[k]['date'])
                    pair_date2.append(data.iloc[p]['date'])
                    pair_text1.append(data.iloc[k]['lemma'])
                    pair_text2.append(data.iloc[p]['lemma'])
                    pair_state1.append(data.iloc[k]['state'])
                    pair_state2.append(data.iloc[p]['state'])
    return pd.DataFrame({'no_reuse':no_reuse, 'pair_index1':pair_index1, 'pair_index2':pair_index2, 
             'pair_date1':pair_date1, 'pair_date2':pair_date2, 
              'pair_text1':pair_text1, 'pair_text2':pair_text2,
                'pair_state1':pair_state1, 'pair_state2':pair_state2})

In [5]:
df=lemmatization(df)

In [6]:
df.columns

Index(['state', 'city', 'date', 'lccn', 'sent', 'title', 'lemma'], dtype='object')

In [7]:
df.to_csv(path+'coolie-token-lemma.csv', index=False)

In [12]:
path='/Users/mstudio/Library/CloudStorage/Box-Box/coolie/sent/'
df=pd.read_csv(path+'/coolie-lemma.csv', converters={'state':literal_eval, 'context':literal_eval, 'city':literal_eval})

In [9]:
df.shape

(125253, 7)

In [10]:
df['lemma_str']=df['lemma'].apply(lambda x: ' '.join(x))

In [12]:
df['5-gram']=df['lemma_str'].apply(lambda x: get_ngrams(x, 5))

In [22]:
reprintdf=reuse_df(df)

100%|██████████| 125253/125253 [23:48:09<00:00,  1.46it/s]  


In [23]:
reprintdf.shape

(96115, 9)

In [25]:
reprintdf.head(2)

,no_reuse,pair_index1,pair_index2,pair_date1,pair_date2,pair_text1,pair_text2,pair_state1,pair_state2
0,3,1,72742,18910826,18910826,"[tho, steamer, namchow, sail, port, chinese, c...","[bteamer, namchow, sail, port, chinese, coolie...",[New York],[New York]
1,3,6,31542,18841025,18841029,"[chinese, general, send, soldier, disguise, co...","[chinese, general, send, soldier, disguise, co...",[Montana],[South Dakota]


In [ ]:
reprintdf.to_csv(path+'reprint.csv', index=False)